The goal of this notebook is to obtain the data required to do the survival analysis for stroke, mi, death and dementia events with the Kaiser population. The analysis is performed using R in a separate notebook.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from microsim.outcome import OutcomeType
from microsim.population_factory import PopulationFactory

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)
dataDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/NOTEBOOKS/DATA"

In [2]:
pd.set_option('future.no_silent_downcasting', True) 

In [3]:
%%time
popSize = 500000
pop = PopulationFactory.get_kaiser_population(n=popSize)

CPU times: user 2min 5s, sys: 18.7 s, total: 2min 23s
Wall time: 2min 15s


In [4]:
%%time
pop.advance(11, nWorkers=5)

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


CPU times: user 2min 29s, sys: 6min 5s, total: 8min 34s
Wall time: 39min 54s


In [5]:
#obtain data for the stroke survival analysis, see figure 1 in Kent2021
strokeInfo = pop.get_outcome_survival_info(outcomesTypeList = [OutcomeType.STROKE],
                                           personFunctionsList = [lambda x: x.get_scd_group(), 
                                                                  lambda x: x.get_wmh_severity_by_modality_group()])
df = pd.DataFrame(strokeInfo, columns=["time","event", "sbiwmhGroup", "severityGroup"])
df.to_csv(dataDir+"/kaiserStrokeValidation13YrTimes.csv", index=False)
df.head()

,time,event,sbiwmhGroup,severityGroup
0,11,0,0,0
1,11,0,0,0
2,8,0,0,0
3,11,0,2,1
4,11,0,0,0


In [6]:
miInfo = pop.get_outcome_survival_info(outcomesTypeList = [OutcomeType.MI],
                                           personFunctionsList = [lambda x: x.get_scd_group(), 
                                                                  lambda x: x.get_wmh_severity_by_modality_group()])
df = pd.DataFrame(miInfo, columns=["time","event", "sbiwmhGroup", "severityGroup"])
df.to_csv(dataDir+"/kaiserMiValidation13YrTimes.csv", index=False)
df.head()

,time,event,sbiwmhGroup,severityGroup
0,11,0,0,0
1,11,0,0,0
2,8,0,0,0
3,11,0,2,1
4,9,1,0,0


In [7]:
#obtain data for the dementia survival analysis, see figure 2 in Kent2023
dementiaInfo = pop.get_outcome_survival_info(outcomesTypeList = [OutcomeType.DEMENTIA],
                                           personFunctionsList = [lambda x: x.get_wmh_severity_by_modality_group(),
                                                                  lambda x: int(x.get_outcome_item_first(OutcomeType.WMH, "sbi")),
                                                                  lambda x: int(x.get_outcome_item_first(OutcomeType.WMH, "wmh"))])
df = pd.DataFrame(dementiaInfo, columns=["time","event", "severityGroup", "sbi", "wmh"])
df.to_csv(dataDir+"/kaiserDementiaValidation13YrTimes.csv", index=False)
df.head()

,time,event,severityGroup,sbi,wmh
0,11,0,0,0,0
1,11,0,0,0,0
2,8,0,0,0,0
3,11,0,1,0,1
4,11,0,0,0,0


In [8]:
deathInfo = pop.get_outcome_survival_info(outcomesTypeList = [OutcomeType.DEATH],
                                           personFunctionsList = [lambda x: x.get_wmh_severity_by_modality_group(),
                                                                  lambda x: int(x.get_outcome_item_first(OutcomeType.WMH, "sbi")),
                                                                  lambda x: int(x.get_outcome_item_first(OutcomeType.WMH, "wmh"))])
df = pd.DataFrame(deathInfo, columns=["time","event", "severityGroup", "sbi", "wmh"])
df.to_csv(dataDir+"/kaiserDeathValidation13YrTimes.csv", index=False)
df.head()

,time,event,severityGroup,sbi,wmh
0,11,0,0,0,0
1,11,0,0,0,0
2,8,1,0,0,0
3,11,0,1,0,1
4,11,0,0,0,0


In [9]:
strokeRatesRef = {1: 36.6, 2: 28.5, 3: 47.4, 0: 8.2, 5:31.2, 6: 13.,  7:34.5, 4: 4.8}
strokeMinCiRef = {1: 34.9, 2: 27.7, 3: 44.5, 0: 8.,  5:28.,  6: 12.4, 7:30.6, 4: 4.5}
strokeMaxCiRef = {1: 38.4, 2: 29.3, 3: 50.5, 0: 8.4, 5:34.6, 6: 13.6, 7:38.7, 4: 5.2}
groupStrings = {1:"CT SBI", 2: "CT WMD", 3: "CT BOTH", 0: "CT NONE", 5:"MRI SBI", 6:"MRI WMD", 7:"MRI BOTH", 4:"MRI NONE"}
strokeRates = pop.get_outcome_incidence_rates_by_scd_and_modality_at_end_of_wave(outcomesTypeList=[OutcomeType.STROKE], wave=3)
strokeRatesList = list()
print("     Group                  Reference     Simulation")
for group in strokeRatesRef.keys():
    strokeRatesList += [ [f"{groupStrings[group]:10} ", 
                                f"{strokeRatesRef[group]:>4.1f} ({strokeMinCiRef[group]:>5.1f} - {strokeMaxCiRef[group]:>4.1f} ) ",
                                f"{strokeRates[group]:<4.1f}" ] ]
    print(f"{groupStrings[group]:>10} " + 
          f"{strokeRatesRef[group]:>10.1f} ({strokeMinCiRef[group]:>5.1f} - {strokeMaxCiRef[group]:>4.1f} ) " +
          f"{strokeRates[group]:>14.1f}")
strokeRatesPd = pd.DataFrame(strokeRatesList, columns=["Group", "Reference", "Simulation"])        
strokeRatesPd.to_csv(dataDir+"/kaiserStrokeValidation4YrRates.csv", index=False)
#strokeRatesPd

     Group                  Reference     Simulation
    CT SBI       36.6 ( 34.9 - 38.4 )           34.0
    CT WMD       28.5 ( 27.7 - 29.3 )           27.1
   CT BOTH       47.4 ( 44.5 - 50.5 )           54.3
   CT NONE        8.2 (  8.0 -  8.4 )            6.9
   MRI SBI       31.2 ( 28.0 - 34.6 )           26.0
   MRI WMD       13.0 ( 12.4 - 13.6 )           12.6
  MRI BOTH       34.5 ( 30.6 - 38.7 )           36.5
  MRI NONE        4.8 (  4.5 -  5.2 )            5.2


In [10]:
dementiaRatesRef = {1:32.8, 2:37.7, 3:51.6, 0:6.7, 5:16.6, 6:9.6, 7:19.1, 4:2.9}
dementiaMinCiRef = {1:31.,  2:36.7, 3:48.3, 0:6.5, 5:14.2, 6:9.1, 7:16.2, 4:2.7}
dementiaMaxCiRef = {1:34.6, 2:38.7, 3:55.1, 0:6.9, 5:19.3, 6:10.1,7:22.4, 4:3.3}
dementiaRates = pop.get_outcome_incidence_rates_by_scd_and_modality_at_end_of_wave(outcomesTypeList=[OutcomeType.DEMENTIA], wave=3)
dementiaRatesList = list()
print("     Group                  Reference     Simulation")
for group in dementiaRatesRef.keys():
    dementiaRatesList += [ [f"{groupStrings[group]:>10} ", 
          f"{dementiaRatesRef[group]:>10.1f} ({dementiaMinCiRef[group]:>5.1f} - {dementiaMaxCiRef[group]:>4.1f} ) ",
          f"{dementiaRates[group]:>14.1f}"] ]
    print(f"{groupStrings[group]:>10} " + 
          f"{dementiaRatesRef[group]:>10.1f} ({dementiaMinCiRef[group]:>5.1f} - {dementiaMaxCiRef[group]:>4.1f} ) " +
          f"{dementiaRates[group]:>14.1f}")
dementiaRatesPd = pd.DataFrame(dementiaRatesList, columns=["Group", "Reference", "Simulation"])        
dementiaRatesPd.to_csv(dataDir+"/kaiserDementiaValidation4YrRates.csv", index=False)

     Group                  Reference     Simulation
    CT SBI       32.8 ( 31.0 - 34.6 )           14.4
    CT WMD       37.7 ( 36.7 - 38.7 )           28.6
   CT BOTH       51.6 ( 48.3 - 55.1 )           60.2
   CT NONE        6.7 (  6.5 -  6.9 )            6.5
   MRI SBI       16.6 ( 14.2 - 19.3 )            5.5
   MRI WMD        9.6 (  9.1 - 10.1 )            9.1
  MRI BOTH       19.1 ( 16.2 - 22.4 )           15.8
  MRI NONE        2.9 (  2.7 -  3.3 )            2.8


In [11]:
miRates = pop.get_outcome_incidence_rates_by_scd_and_modality_at_end_of_wave(outcomesTypeList=[OutcomeType.MI], wave=3)
#print("     Group                  Reference     Simulation")
print("     Group     Simulation")
miRatesList = list()
for group in groupStrings.keys():
    miRatesList += [ [f"{groupStrings[group]:>10} ",  
                      f"{miRates[group]:>14.1f}"] ]
    print(f"{groupStrings[group]:>10} " + 
          f"{miRates[group]:>14.1f}")
miRatesPd = pd.DataFrame(miRatesList, columns=["Group", "Simulation"])        
miRatesPd.to_csv(dataDir+"/kaiserMiValidation4YrRates.csv", index=False)

     Group     Simulation
    CT SBI           36.1
    CT WMD           23.2
   CT BOTH           41.9
   CT NONE            7.0
   MRI SBI           35.3
   MRI WMD           12.8
  MRI BOTH           39.0
  MRI NONE            6.1


In [12]:
deathRatesRef = {1:61.5, 2: 63.8, 3: 84.9, 0:18.2, 5:49.2, 6:28.5, 7:53.7, 4:14.}
deathMinCiRef = {1:59.1, 2:62.6,  3: 80.9, 0:17.8, 5:45.1, 6:27.6, 7:48.8, 4:13.4}
deathMaxCiRef = {1:63.9, 2:65.1,  3:89.2,  0:18.5, 5:53.6, 6:29.4, 7:59.0, 4:14.6}
deathRates = pop.get_outcome_incidence_rates_by_scd_and_modality_at_end_of_wave(outcomesTypeList=[OutcomeType.DEATH], wave=3)
deathRatesList = list()
print("     Group                  Reference     Simulation")
for group in deathRatesRef.keys():
    deathRatesList += [ [f"{groupStrings[group]:>10} ", 
                         f"{deathRatesRef[group]:>10.1f} ({deathMinCiRef[group]:>5.1f} - {deathMaxCiRef[group]:>4.1f} ) ",
                         f"{deathRates[group]:>14.1f}"] ]
    print(f"{groupStrings[group]:>10} " + 
          f"{deathRatesRef[group]:>10.1f} ({deathMinCiRef[group]:>5.1f} - {deathMaxCiRef[group]:>4.1f} ) " +
          f"{deathRates[group]:>14.1f}")
deathRatesPd = pd.DataFrame(deathRatesList, columns=["Group", "Reference", "Simulation"])        
deathRatesPd.to_csv(dataDir+"/kaiserDeathValidation4YrRates.csv", index=False)

     Group                  Reference     Simulation
    CT SBI       61.5 ( 59.1 - 63.9 )           42.5
    CT WMD       63.8 ( 62.6 - 65.1 )           52.6
   CT BOTH       84.9 ( 80.9 - 89.2 )           95.2
   CT NONE       18.2 ( 17.8 - 18.5 )           18.9
   MRI SBI       49.2 ( 45.1 - 53.6 )           37.0
   MRI WMD       28.5 ( 27.6 - 29.4 )           30.5
  MRI BOTH       53.7 ( 48.8 - 59.0 )           71.9
  MRI NONE       14.0 ( 13.4 - 14.6 )           13.4
